**RANDOM FOREST**

In [20]:
!pip install scikit-optimize

  Using cached scikit_optimize-0.10.2-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached pyaml-24.7.0-py3-none-any.whl.metadata (11 kB)
Using cached scikit_optimize-0.10.2-py2.py3-none-any.whl (107 kB)
Using cached pyaml-24.7.0-py3-none-any.whl (24 kB)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import pickle

# Load the datasets
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')

# Filter for surface data in tikob dataset
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

# Columns to exclude
columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']

# Filter columns for both datasets
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

# Define feature columns and target column
feature_columns = [
    'pH', 'DO (mg/L)', 'TDS (mg/L)', 'Salinity (ppt)', 'Cond (uS/cm)', 'Temp (°C)', 'TSS (mg/L)', 
    'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)', 'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 
    'Chlorophyll (ug/L)', 'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)', 
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

# Extract features and target from both datasets
X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]
X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Normalize features
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

# Train-test split for full dataset
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# Hyperparameter tuning using Bayesian Optimization on RandomForestRegressor
param_grid_rf = {
    'n_estimators': (100, 500),  # Reduced upper limit
    'max_depth': (1, 30),  # Reduced upper limit
    'min_samples_split': (2, 10),  # Reduced upper limit
    'min_samples_leaf': (1, 10),  # Reduced upper limit
    'bootstrap': [True, False]
}

# Initialize BayesSearchCV
bayes_search_rf = BayesSearchCV(estimator=RandomForestRegressor(random_state=42),
                                search_spaces=param_grid_rf,
                                n_iter=30,  # Reduced number of iterations
                                cv=3,  # Reduced number of cross-validation folds
                                n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the model
bayes_search_rf.fit(X_train_full, y_train_full)

# Best parameters from Bayesian optimization
best_params_rf = bayes_search_rf.best_params_

# Train the model with the best parameters
rf_optimized = RandomForestRegressor(**best_params_rf, random_state=42)
rf_optimized.fit(X_train_full, y_train_full)

# Save the model to a pickle file
with open('rf_optimized_model.pkl', 'wb') as model_file:
    pickle.dump(rf_optimized, model_file)

# Load the model from the pickle file (for demonstration purposes)
with open('rf_optimized_model.pkl', 'rb') as model_file:
    loaded_rf_model = pickle.load(model_file)

# Cross-validation to check the performance consistency
cv_scores = cross_val_score(loaded_rf_model, X_train_full, y_train_full, cv=3, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores.mean())

# Evaluate on test set
test_predictions_optimized = loaded_rf_model.predict(X_test_full)
test_mse_optimized = mean_squared_error(y_test_full, test_predictions_optimized)
test_rmse_optimized = np.sqrt(test_mse_optimized)
test_mae_optimized = mean_absolute_error(y_test_full, test_predictions_optimized)
test_r2_optimized = r2_score(y_test_full, test_predictions_optimized)
test_mape_optimized = np.mean(np.abs((y_test_full - test_predictions_optimized) / y_test_full)) * 100

print('Optimized Test Results with RandomForestRegressor:')
print('MSE:', test_mse_optimized)
print('RMSE:', test_rmse_optimized)
print('MAE:', test_mae_optimized)
print('R^2:', test_r2_optimized)
print('MAPE:', test_mape_optimized, '%')
print('Cross-validated RMSE:', cv_rmse)

# Feature importance analysis
feature_importances = loaded_rf_model.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]
print("Feature ranking:")
for i in range(X_combined.shape[1]):
    print(f"{i + 1}. feature {sorted_indices[i]} ({feature_importances[sorted_indices[i]]})")

# Re-train model using only the top features
top_n = 10  # Number of top features to use
top_features = sorted_indices[:top_n]
X_combined_top = X_combined_scaled[:, top_features]

X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(X_combined_top, y_combined, test_size=0.2, random_state=42)

# Re-train the optimized model using only top features
rf_optimized_top = RandomForestRegressor(**best_params_rf, random_state=42)
rf_optimized_top.fit(X_train_top, y_train_top)

# Save the model trained on top features to a pickle file
with open('rf_optimized_top_features_model.pkl', 'wb') as model_file:
    pickle.dump(rf_optimized_top, model_file)

# Load the model from the pickle file (for demonstration purposes)
with open('rf_optimized_top_features_model.pkl', 'rb') as model_file:
    loaded_rf_model_top = pickle.load(model_file)

# Evaluate on test set with top features
test_predictions_optimized_top = loaded_rf_model_top.predict(X_test_top)
test_mse_optimized_top = mean_squared_error(y_test_top, test_predictions_optimized_top)
test_rmse_optimized_top = np.sqrt(test_mse_optimized_top)
test_mae_optimized_top = mean_absolute_error(y_test_top, test_predictions_optimized_top)
test_r2_optimized_top = r2_score(y_test_top, test_predictions_optimized_top)
test_mape_optimized_top = np.mean(np.abs((y_test_top - test_predictions_optimized_top) / y_test_top)) * 100

print('Optimized Test Results with RandomForestRegressor (Top Features):')
print('MSE:', test_mse_optimized_top)
print('RMSE:', test_rmse_optimized_top)
print('MAE:', test_mae_optimized_top)
print('R^2:', test_r2_optimized_top)
print('MAPE:', test_mape_optimized_top, '%')


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

**SVR**

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
import pickle

# Load the datasets
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')

# Filter for surface data in tikob dataset
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

# Columns to exclude
columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']

# Filter columns for both datasets
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

# Define feature columns and target column
feature_columns = [
    'pH', 'DO (mg/L)', 'TDS (mg/L)', 'Salinity (ppt)', 'Cond (uS/cm)', 'Temp (°C)', 'TSS (mg/L)', 
    'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)', 'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 
    'Chlorophyll (ug/L)', 'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)', 
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

# Extract features and target from both datasets
X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]
X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Scale the features
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

# First tier: small subset for initial parameter tuning
X_small, X_rest, y_small, y_rest = train_test_split(X_combined_scaled, y_combined, test_size=0.9, random_state=42)

# Train-test split for small subset
X_train_small, X_val_small, y_train_small, y_val_small = train_test_split(X_small, y_small, test_size=0.2, random_state=42)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'svr__C': [0.01, 0.1, 1, 10, 100],
    'svr__epsilon': [0.001, 0.01, 0.1, 1],
    'poly__degree': [1, 2, 3]
}
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('svr', SVR(kernel='rbf'))
])

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train_small, y_train_small)

# Train SVR model on small subset with best parameters
svr_small = grid_search.best_estimator_
svr_small.fit(X_train_small, y_train_small)

# Evaluate on validation set
val_predictions_small = svr_small.predict(X_val_small)
val_mse_small = mean_squared_error(y_val_small, val_predictions_small)
val_rmse_small = np.sqrt(val_mse_small)
val_mae_small = mean_absolute_error(y_val_small, val_predictions_small)
val_r2_small = r2_score(y_val_small, val_predictions_small)
val_mape_small = np.mean(np.abs((y_val_small - val_predictions_small) / y_val_small)) * 100

print('Validation Results on Small Subset:')
print('MSE:', val_mse_small)
print('RMSE:', val_rmse_small)
print('MAE:', val_mae_small)
print('R^2:', val_r2_small)
print('MAPE:', val_mape_small, '%')

# Second tier: larger subset for more refined training
X_large, X_rest, y_large, y_rest = train_test_split(X_combined_scaled, y_combined, test_size=0.5, random_state=42)

# Train-test split for large subset
X_train_large, X_val_large, y_train_large, y_val_large = train_test_split(X_large, y_large, test_size=0.2, random_state=42)

# Train SVR model on large subset with best parameters
svr_large = grid_search.best_estimator_
svr_large.fit(X_train_large, y_train_large)

# Evaluate on validation set
val_predictions_large = svr_large.predict(X_val_large)
val_mse_large = mean_squared_error(y_val_large, val_predictions_large)
val_rmse_large = np.sqrt(val_mse_large)
val_mae_large = mean_absolute_error(y_val_large, val_predictions_large)
val_r2_large = r2_score(y_val_large, val_predictions_large)
val_mape_large = np.mean(np.abs((y_val_large - val_predictions_large) / y_val_large)) * 100

print('Validation Results on Large Subset:')
print('MSE:', val_mse_large)
print('RMSE:', val_rmse_large)
print('MAE:', val_mae_large)
print('R^2:', val_r2_large)
print('MAPE:', val_mape_large, '%')

# Third tier: full dataset for final training and testing
# Train-test split for full dataset
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_combined_scaled, y_combined, test_size=0.2, random_state=42)

# Train SVR model on full dataset with best parameters
svr_full = grid_search.best_estimator_
svr_full.fit(X_train_full, y_train_full)

# Evaluate on test set
test_predictions_full = svr_full.predict(X_test_full)
test_mse_full = mean_squared_error(y_test_full, test_predictions_full)
test_rmse_full = np.sqrt(test_mse_full)
test_mae_full = mean_absolute_error(y_test_full, test_predictions_full)
test_r2_full = r2_score(y_test_full, test_predictions_full)
test_mape_full = np.mean(np.abs((y_test_full - test_predictions_full) / y_test_full)) * 100

print('Test Results on Full Dataset:')
print('MSE:', test_mse_full)
print('RMSE:', test_rmse_full)
print('MAE:', test_mae_full)
print('R^2:', test_r2_full)
print('MAPE:', test_mape_full, '%')

# Save the trained model to a pickle file
with open('trained_svr_model.pkl', 'wb') as f:
    pickle.dump(svr_full, f)

print("Model saved to trained_svr_model.pkl")


Validation Results on Small Subset:
MSE: 1.0056481446537993
RMSE: 1.0028200958565794
MAE: 0.8141328563814624
R^2: 0.16289726012742034
MAPE: 52.539823930365706 %
Validation Results on Large Subset:
MSE: 0.7889408430331978
RMSE: 0.888223419547806
MAE: 0.5904872332040191
R^2: 0.6806536032580996
MAPE: 35.22313912378321 %
Test Results on Full Dataset:
MSE: 0.3861713071978852
RMSE: 0.6214268317331375
MAE: 0.4387562303294686
R^2: 0.8463871521904118
MAPE: 29.02174693916996 %
Model saved to trained_svr_model.pkl


**MLR**

In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the datasets
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')

# Filter for surface data in tikob dataset
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

# Columns to exclude
columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']

# Filter columns for both datasets
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

# Define feature columns and target column
feature_columns = [
    'pH', 'DO (mg/L)', 'TDS (mg/L)', 'Salinity (ppt)', 'Cond (uS/cm)', 'Temp (°C)', 'TSS (mg/L)', 
    'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)', 'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 
    'Chlorophyll (ug/L)', 'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)', 
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

# Extract features and target from both datasets
X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]
X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Feature Selection: Check for highly correlated features and drop them
correlation_matrix = X_combined.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]
print("Highly correlated features:", high_corr_features)
X_combined_filtered = X_combined.drop(columns=high_corr_features)

# Train-test split for full dataset
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_combined_filtered, y_combined, test_size=0.2, random_state=42)

# Random Forest Regression
rf = RandomForestRegressor(random_state=42)
parameters = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_regressor = GridSearchCV(rf, parameters, scoring='r2', cv=5, n_jobs=-1)

# Fit the Random Forest model
rf_regressor.fit(X_train_full, y_train_full)

# Best parameters
best_params_rf = rf_regressor.best_params_
print("Best parameters for Random Forest regression:", best_params_rf)

# Save the model to a .pkl file
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf_regressor, file)

# Predict on the test set
test_predictions_rf = rf_regressor.predict(X_test_full)
test_mse_rf = mean_squared_error(y_test_full, test_predictions_rf)
test_rmse_rf = np.sqrt(test_mse_rf)
test_mae_rf = mean_absolute_error(y_test_full, test_predictions_rf)
test_r2_rf = r2_score(y_test_full, test_predictions_rf)
test_mape_rf = np.mean(np.abs((y_test_full - test_predictions_rf) / y_test_full)) * 100

print('Test Results with Random Forest Regression:')
print('MSE:', test_mse_rf)
print('RMSE:', test_rmse_rf)
print('MAE:', test_mae_rf)
print('R^2:', test_r2_rf)
print('MAPE:', test_mape_rf, '%')


Highly correlated features: ['TN (ppm)', 'TP (ppm)']
Best parameters for Random Forest regression: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Test Results with Random Forest Regression:
MSE: 0.8120282453938545
RMSE: 0.9011260984977932
MAE: 0.5622765057251262
R^2: 0.6769879870622968
MAPE: 37.96740898199653 %


**MLP**

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load the datasets
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')

# Filter for surface data in tikob dataset
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

# Columns to exclude
columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']

# Filter columns for both datasets
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

# Define feature columns and target column
feature_columns = [
    'pH', 'DO (mg/L)', 'TDS (mg/L)', 'Salinity (ppt)', 'Cond (uS/cm)', 'Temp (°C)', 'TSS (mg/L)', 
    'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)', 'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 
    'Chlorophyll (ug/L)', 'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)', 
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

# Extract features and target from both datasets
X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]
X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Train-test split for full dataset
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Define a pipeline with standard scaling and MLPRegressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPRegressor(max_iter=1000, random_state=42))
])

# Define a parameter grid for GridSearchCV
param_grid = {
    'mlp__hidden_layer_sizes': [(50, 50), (100, 100), (100, 50, 100)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__solver': ['adam', 'lbfgs'],
    'mlp__alpha': [0.0001, 0.001, 0.01],
    'mlp__learning_rate': ['constant', 'adaptive']
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Evaluate on test set
test_predictions = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)
test_mape = np.mean(np.abs((y_test - test_predictions) / y_test)) * 100

print('Test Results with Best Model:')
print('MSE:', test_mse)
print('RMSE:', test_rmse)
print('MAE:', test_mae)
print('R^2:', test_r2)
print('MAPE:', test_mape, '%')
print('Best Parameters:', grid_search.best_params_)


Test Results with Best Model:
MSE: 0.3100669079910257
RMSE: 0.5568365181909549
MAE: 0.4088966799670373
R^2: 0.8766602803982847
MAPE: 31.649853842599118 %
Best Parameters: {'mlp__activation': 'tanh', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (100, 50, 100), 'mlp__learning_rate': 'constant', 'mlp__solver': 'adam'}
